# General relativity using symbolic computer algebra
## Introduction to GraviPy

The popular programming language [Python](https://www.python.org) provides a package for symbolic computations called [SymPy](https://www.sympy.org). In 2014, Wojciech Czaja started writing the package GraviPy which extends SymPy with functionality useful in doing symbolic calculations in general relativity. The package is available from the [Python Package Index](https://pypi.python.org) (PyPI) as [GraviPy 0.2.0](https://pypi.org/project/gravipy/). In this notebook, we will give an introduction to these packages to the extent needed in the following notebooks.

#### Installation

Probably the easiest way to set up a Python environment for the use with GraviPy is to start with the [Anaconda distribution](https://www.anaconda.org) which is a popular distribution in scientific environments. It provides conda, a package and environment management system allowing us to easily setup a Python environment with all the packages needed to run the notebooks in this collection.

Assuming that you have the Anaconda distribution installed and its binaries in your path, follow these steps to install an environment for use with these notebooks.

1. Create a new environment. Instead of `gravipy` you can use also another name for the environment as long as it does not conflict with already existing environments.
```
conda create --name gravipy
```
**Important note**: For Python 3.6+, GraviPy 0.2.0+ should be used.

1. Activate the new environment. If you have chosen a different name for the environment, you should use it here as well.
```
conda activate gravipy
```
1. Install the SymPy package.
```
conda install sympy
```
1. Install the GraviPy Package from PyPI. At the time of writing, no `conda` package for GraviPy was available, so that we are obliged to install it with `pip`. Note that in the following command the package name is required, not the name of the environment chosen above.
```
pip install gravipy
```
1. In order to run the notebooks in this environment, the Jupyter package needs to be installed as well.
```
conda install jupyter
```
1. After finishing work with the notebooks, you may deactivate the environment.
```
conda deactivate
```

Alternatively, the `requirements.txt` file specifies the requirements to set up a Python virtual environment with GraviPy 0.2.0.

#### Getting started

Before GraviPy can be used, it needs to be imported. GraviPy 0.2.0 itself will import some functionality of `sympy` but not the full namespace as was the case for GraviPy 0.1.0. In the third line, we call SymPy's `init_printing` function which will make sure that the visual representation of results will be optimized.

In [ ]:
from gravipy.tensorial import *
import sympy
sympy.init_printing()

#### Metric tensor
For our first steps, we start by considering a two-dimensional sphere embedded in three-dimensional space. Only later will we move to four-dimensional space-time. We begin by introducing symbols and define two of the symbols as coordinates.

In [ ]:
r, theta, phi = symbols(r'r theta phi')
x = Coordinates('x', [theta, phi])

On the left-hand side of the first line, we define three variables `r`, `theta` and `phi`. There visual representation is given in the argument of `symbols`, where the backslash refers to the TeX syntax of greek letters. Note the 'r' in front of the string, defining it as a raw string. Otherwise, `\t` would be interpreted as tab character.

In the second line, we define a `Coordinates` object which possesses an attribute `c` containing the coordinates:

In [ ]:
x.c

At this point, the contravariant components are defined, which in GraviPy are indexed by negative indices:

In [ ]:
x.components

In [ ]:
x(-1), x(-2)

The covariant components indexed by a positive index are defined only after the associated metric tensor has been defined.

**Note**: With a background in general relativity and/or Python, it may seem odd that numbering in GraviPy starts at 1 and not at 0. However, since the sign is used to distinguish between contravariant and covariant components, the index 0 is not useful.

In order to define the metric tensor `g`, we first introduce a diagonal matrix with the entries appropriate for a sphere of radius `r`.

In [ ]:
metric = diag(r**2, r**2*sin(theta)**2)
metric

At this point, we just have a SymPy matrix which now needs to be converted into a metric tensor. The first argument gives the symbol used to represent the metric tensor, the second  argument gives the coordinates, and the last argument is the matrix just introduced which defines the components of the metric tensor.  

In [ ]:
g = MetricTensor('g', x, metric)

We cn obtain all components of the metric tensor $g_{ij}$ as follows:

In [ ]:
g(All, All)

Correspondingly, its inverse $g^{ij}$ is obtained by using negative indices.

In [ ]:
g(-All, -All)

The relation
$$g_{ij}g^{jk} = g_i^k = \delta_i^k$$
signifies that $g^{ij}$ is the inverse of $g_{ij}$ which can now easily be verified by a matrix multiplication.

In [ ]:
g(All, All)*g(-All, -All)

The mixed representation $g_i^j$ can be obtained by using a negative sign for only one of the components.

In [ ]:
g(-All, All)

or

In [ ]:
g(All, -All)

Once the metric has been defined, also covariant coordinates are defined.

In [ ]:
x.components

#### Christoffel symbol

From the metric tensor, we can define the Christoffel symbol of the first kind
$$\Gamma_{ijk} = \frac{1}{2}\left(\partial_j g_{ik}+\partial_k g_{ji}-\partial_i g_{jk}\right)$$

In [ ]:
Gamma = Christoffel(r'\Gamma', g)

In [ ]:
Gamma(All, All, All)

We thus find two different nonvanishing components of the Christoffel symbol
$$\Gamma_{122} = -\frac{r^2}{2}\sin(2\theta)$$
and
$$\Gamma_{212} = \Gamma_{221} = \frac{r^2}{2}\sin(\theta).$$

The Christoffel symbol of the second kind
$$\Gamma^i_{jk} = \frac{1}{2}g^{im}\left(\partial_j g_{mk}+\partial_k g_{jm}-\partial_m g_{jk}\right)$$
is obtained as

In [ ]:
Gamma(-All, All, All)

with the two different components
$$\Gamma^1_{22} = -\frac{1}{2}\sin(2\theta)$$
and
$$\Gamma^2_{12} = \Gamma^2_{21} = \cot(\theta)$$
where the last expression can be verified with the help of SymPy's `simplify` method.

In [ ]:
Gamma(-2, 1, 2).simplify()

#### Riemann curvature tensor

The Riemann curvature tensor is obtained from the metric and the Christoffels symbols of the second kind as
$$R_{ijkm} = \frac{1}{2}\left(
    \frac{\partial^2g_{im}}{\partial x^j\partial x^k}
    +\frac{\partial^2g_{jk}}{\partial x^i\partial x^m}
    -\frac{\partial^2g_{ik}}{\partial x^j\partial x^m}
    -\frac{\partial^2g_{jm}}{\partial x^i\partial x^k}
\right)
+g_{np}(\Gamma^n_{jk}\Gamma^p_{im}-\Gamma^n_{jm}\Gamma^p_{ik})
$$

In [ ]:
Rm = Riemann('Rm', g)

In [ ]:
Rm(All, All, All, All)

This Riemann curvature tensor contains only one non-trivial entry as all others can be obtained by means of the symmetry properties
$$R_{1212} = -R_{2112} = -R_{1221} = R_{2121} = r^2\sin(2\theta).$$

In [ ]:
Rm(1, 2, 1, 2), -Rm(2, 1, 1, 2), -Rm(1, 2, 2, 1), Rm(2, 1, 2, 1)

All other components of the Riemann curvature tensor in two dimensions vanish because of symmetry reasons.

#### Ricci tensor, Ricci scalar, and Einstein tensor

The Ricci tensor appearing in Einstein's field equation of general relativity is defined as contraction of the Riemann curvature tensor
$$R_{ij} = R^k_{ikj}.$$
A further contraction yields the Ricci scalar
$$R = R^i_i.$$
The Einstein tensor is then defined as
$$G^{ij} = R^{ij}-\frac{1}{2}g^{ij}R.$$
All these quantities are available with GraviPy.

In [ ]:
Ri = Ricci('Ri', g)

In [ ]:
Ri(All, All)

In [ ]:
Ri.scalar()

In [ ]:
G = Einstein('G', Ri)

In [ ]:
G(-All, -All)